In [1]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('cna_category_news.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(article_word, article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

Inference: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]

Tokenize OK!
CPU times: total: 43.5 s
Wall time: 24.7 s


In [2]:
## Read it out 讀出看看
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')
df.head(1)


,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,暫無,暫無,"[('醫院', 24), ('離島', 7), ('石崇良', 7), ('護理', 5),...","['為', '解決', '急診', '壅塞', '，', '健保署', '提出', '增加'...","['健保署', '標準', '護理', '人員', '離島', '責任', '醫院', '醫...","[NerToken(word='今天', ner='DATE', idx=(58, 60))...","[('為', 'P'), ('解決', 'VC'), ('急診', 'VB'), ('壅塞'...",/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...


In [3]:
import pandas as pd
import numpy

In [4]:
df = pd.read_csv('cna_category_news.csv', sep='|')

In [5]:
df.shape

(20, 7)

In [6]:
df.head()

,item_id,date,category,title,content,link,photo_link
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
1,ahel_20250312_2,2025-03-12,生活,今彩539第114064期 頭獎槓龜,今彩539第114064期開獎，中獎號碼06、14、27、30、33；派彩結果，頭獎槓龜。貳...,/news/ahel/202503120395.aspx,NaN
2,ahel_20250312_3,2025-03-12,生活,今彩539第114064期開獎,今彩539第114064期開獎，中獎號碼06、14、27、30、33。3星彩中獎號碼760，...,/news/ahel/202503120374.aspx,NaN
3,ahel_20250312_4,2025-03-12,生活,邱泰源與醫療工會開會 達擬跨科支援指引等共識,近日急診壅塞再掀護理人力不足等待解難題，衛生福利部長邱泰源今天與醫療相關工會代表開會，裁示將...,/news/ahel/202503120365.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
4,aopl_20250312_1,2025-03-12,國際,英國鋼鋁遭美徵收25%關稅 英相施凱爾透露不排除反制,美國向全球徵收高達25%的鋼、鋁進口關稅，今天生效，英國未獲豁免。英國首相施凱爾今天在國會答...,/news/aopl/202503120410.aspx,NaN


In [7]:
df.content[0:5]

0    為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...
1    今彩539第114064期開獎，中獎號碼06、14、27、30、33；派彩結果，頭獎槓龜。貳...
2    今彩539第114064期開獎，中獎號碼06、14、27、30、33。3星彩中獎號碼760，...
3    近日急診壅塞再掀護理人力不足等待解難題，衛生福利部長邱泰源今天與醫療相關工會代表開會，裁示將...
4    美國向全球徵收高達25%的鋼、鋁進口關稅，今天生效，英國未獲豁免。英國首相施凱爾今天在國會答...
Name: content, dtype: object

In [8]:
df['content'][0:5]

0    為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...
1    今彩539第114064期開獎，中獎號碼06、14、27、30、33；派彩結果，頭獎槓龜。貳...
2    今彩539第114064期開獎，中獎號碼06、14、27、30、33。3星彩中獎號碼760，...
3    近日急診壅塞再掀護理人力不足等待解難題，衛生福利部長邱泰源今天與醫療相關工會代表開會，裁示將...
4    美國向全球徵收高達25%的鋼、鋁進口關稅，今天生效，英國未獲豁免。英國首相施凱爾今天在國會答...
Name: content, dtype: object

In [9]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [10]:
# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

In [11]:
%%time
# It takes time.
tokens = ws(df.content)

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]

CPU times: total: 6.92 s
Wall time: 1.29 s


In [12]:
len(tokens)

20

In [13]:
len(tokens[0])

644

In [14]:
tokens[0]

['為',
 '解決',
 '急診',
 '壅塞',
 '，',
 '健保署',
 '提出',
 '增加',
 '急診',
 '及',
 '重',
 '症',
 '支付',
 '標準',
 '、',
 '給',
 '急診',
 '護理',
 '人員',
 '加薪',
 '、',
 '強化',
 '離島',
 '急救',
 '責任',
 '醫院',
 '等',
 '4',
 '招解',
 '方',
 '，',
 '醫院',
 '總額',
 '研商會',
 '今天',
 '通過',
 '，',
 '最快',
 '5月',
 '上路',
 '，',
 '年',
 '挹注',
 '逾',
 '新台幣',
 '4',
 '2億',
 '元',
 '。',
 '急診',
 '近來',
 '面臨',
 '前所',
 '未',
 '有',
 '壅塞',
 '，',
 '衛生',
 '福利部',
 '中央',
 '健康',
 '保險署',
 '今天',
 '在',
 '醫療',
 '給付',
 '費用',
 '醫院',
 '總額',
 '研商',
 '議事會',
 '討論解',
 '方',
 '，',
 '包含',
 '急診',
 '支付標準',
 '與',
 '留',
 '觀床',
 '支付',
 '、',
 '急救',
 '責任',
 '醫院',
 '加',
 '護',
 '病房',
 '給付',
 '、',
 '強化',
 '離島',
 '急救',
 '責任',
 '醫院',
 '給付與',
 '急性',
 '一般',
 '病床',
 '護理費',
 '調升',
 '等',
 '4',
 '案',
 '。',
 '健保署長',
 '石崇良',
 '今天',
 '晚間',
 '接受',
 '媒體',
 '聯訪',
 '表示',
 '，',
 '與',
 '解決',
 '急診',
 '壅塞',
 '有關',
 '的',
 '4',
 '項',
 '提案',
 '全數',
 '通過',
 '，',
 '合計',
 '挹注',
 '42.35億元',
 '，',
 '將',
 '儘速',
 '送入',
 '共',
 '擬會',
 '與',
 '健保會',
 '，',
 '拚',
 '5月',
 '起',
 '開始',
 '執行',
 '。',
 '這',
 '次',
 '不',
 '是

In [15]:
tokens[0][0]

'為'

In [16]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 4/4 [00:02<00:00,  1.91it/s]

CPU times: total: 20.4 s
Wall time: 2.17 s


In [17]:
len(tokens_pos)

20

In [18]:
len(tokens_pos[0])

644

In [19]:
tokens_pos[0]

['P',
 'VC',
 'VB',
 'VH',
 'COMMACATEGORY',
 'Nc',
 'VC',
 'VHC',
 'VB',
 'Caa',
 'Na',
 'Na',
 'VD',
 'Na',
 'PAUSECATEGORY',
 'P',
 'Nv',
 'Na',
 'Na',
 'VB',
 'PAUSECATEGORY',
 'VHC',
 'Nc',
 'Nv',
 'Na',
 'Nc',
 'Cab',
 'Neu',
 'Nf',
 'D',
 'COMMACATEGORY',
 'Nc',
 'Na',
 'Na',
 'Nd',
 'VC',
 'COMMACATEGORY',
 'Dfa',
 'Nd',
 'VA',
 'COMMACATEGORY',
 'Nf',
 'VC',
 'VJ',
 'Na',
 'Neu',
 'Neu',
 'Nf',
 'PERIODCATEGORY',
 'VB',
 'D',
 'VK',
 'VH',
 'D',
 'V_2',
 'VH',
 'COMMACATEGORY',
 'Na',
 'Na',
 'Nc',
 'VH',
 'Na',
 'Nd',
 'P',
 'VC',
 'VD',
 'Na',
 'Nc',
 'Na',
 'VE',
 'Na',
 'VE',
 'Na',
 'COMMACATEGORY',
 'VJ',
 'Nv',
 'VD',
 'Caa',
 'VC',
 'VC',
 'VD',
 'PAUSECATEGORY',
 'Nv',
 'Na',
 'Nc',
 'VC',
 'VC',
 'Nc',
 'VD',
 'PAUSECATEGORY',
 'VHC',
 'Nc',
 'Nv',
 'Nv',
 'Nc',
 'VD',
 'A',
 'A',
 'Na',
 'Na',
 'VHC',
 'Cab',
 'Neu',
 'Na',
 'PERIODCATEGORY',
 'Na',
 'Nb',
 'Nd',
 'Nd',
 'VC',
 'Na',
 'VC',
 'VE',
 'COMMACATEGORY',
 'P',
 'VC',
 'VB',
 'VH',
 'VJ',
 'DE',
 'Neu',
 '

In [20]:
for article_word, article_pos in zip(tokens,tokens_pos):
    print(article_word)
    print(article_pos)

['為', '解決', '急診', '壅塞', '，', '健保署', '提出', '增加', '急診', '及', '重', '症', '支付', '標準', '、', '給', '急診', '護理', '人員', '加薪', '、', '強化', '離島', '急救', '責任', '醫院', '等', '4', '招解', '方', '，', '醫院', '總額', '研商會', '今天', '通過', '，', '最快', '5月', '上路', '，', '年', '挹注', '逾', '新台幣', '4', '2億', '元', '。', '急診', '近來', '面臨', '前所', '未', '有', '壅塞', '，', '衛生', '福利部', '中央', '健康', '保險署', '今天', '在', '醫療', '給付', '費用', '醫院', '總額', '研商', '議事會', '討論解', '方', '，', '包含', '急診', '支付標準', '與', '留', '觀床', '支付', '、', '急救', '責任', '醫院', '加', '護', '病房', '給付', '、', '強化', '離島', '急救', '責任', '醫院', '給付與', '急性', '一般', '病床', '護理費', '調升', '等', '4', '案', '。', '健保署長', '石崇良', '今天', '晚間', '接受', '媒體', '聯訪', '表示', '，', '與', '解決', '急診', '壅塞', '有關', '的', '4', '項', '提案', '全數', '通過', '，', '合計', '挹注', '42.35億元', '，', '將', '儘速', '送入', '共', '擬會', '與', '健保會', '，', '拚', '5月', '起', '開始', '執行', '。', '這', '次', '不', '是', '只', '挹注', '，', '還', '有', '附帶', '任務', '指標', '，', '共同', '指標', '是', '醫院', '必', '須', '落', '實', '醫護', '人員', '加薪', '，', '其他', '還', '有', '急診', '壅塞', '

In [21]:
word_pos_pair = [list(zip(article_word,article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

In [22]:
len(word_pos_pair)

20

In [23]:
word_pos_pair

[[('為', 'P'),
  ('解決', 'VC'),
  ('急診', 'VB'),
  ('壅塞', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('健保署', 'Nc'),
  ('提出', 'VC'),
  ('增加', 'VHC'),
  ('急診', 'VB'),
  ('及', 'Caa'),
  ('重', 'Na'),
  ('症', 'Na'),
  ('支付', 'VD'),
  ('標準', 'Na'),
  ('、', 'PAUSECATEGORY'),
  ('給', 'P'),
  ('急診', 'Nv'),
  ('護理', 'Na'),
  ('人員', 'Na'),
  ('加薪', 'VB'),
  ('、', 'PAUSECATEGORY'),
  ('強化', 'VHC'),
  ('離島', 'Nc'),
  ('急救', 'Nv'),
  ('責任', 'Na'),
  ('醫院', 'Nc'),
  ('等', 'Cab'),
  ('4', 'Neu'),
  ('招解', 'Nf'),
  ('方', 'D'),
  ('，', 'COMMACATEGORY'),
  ('醫院', 'Nc'),
  ('總額', 'Na'),
  ('研商會', 'Na'),
  ('今天', 'Nd'),
  ('通過', 'VC'),
  ('，', 'COMMACATEGORY'),
  ('最快', 'Dfa'),
  ('5月', 'Nd'),
  ('上路', 'VA'),
  ('，', 'COMMACATEGORY'),
  ('年', 'Nf'),
  ('挹注', 'VC'),
  ('逾', 'VJ'),
  ('新台幣', 'Na'),
  ('4', 'Neu'),
  ('2億', 'Neu'),
  ('元', 'Nf'),
  ('。', 'PERIODCATEGORY'),
  ('急診', 'VB'),
  ('近來', 'D'),
  ('面臨', 'VK'),
  ('前所', 'VH'),
  ('未', 'D'),
  ('有', 'V_2'),
  ('壅塞', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('衛生', 'Na')

In [24]:
a=[1,2,3]
b=['a','b','c']
ab=zip(a,b)
list(ab)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [25]:
# list(zip(w,p)) for w, p in zip(tokens, tokens_pos)
a=[[1,2],[3,4],[5,6]]
b=[['a','b'],['c','d'],['e','f']]
ab=zip(a,b)
list(ab)


[([1, 2], ['a', 'b']), ([3, 4], ['c', 'd']), ([5, 6], ['e', 'f'])]

In [26]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)

[1, 2] with ['a', 'b']
[3, 4] with ['c', 'd']
[5, 6] with ['e', 'f']


In [27]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)
    print(list(zip(item1, item2)))
    print('---')

[1, 2] with ['a', 'b']
[(1, 'a'), (2, 'b')]
---
[3, 4] with ['c', 'd']
[(3, 'c'), (4, 'd')]
---
[5, 6] with ['e', 'f']
[(5, 'e'), (6, 'f')]
---


In [28]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]

CPU times: total: 8.47 s
Wall time: 1.01 s


In [29]:
entity_list[0]

[NerToken(word='今天', ner='DATE', idx=(58, 60)),
 NerToken(word='5月', ner='DATE', idx=(65, 67)),
 NerToken(word='42億元', ner='MONEY', idx=(77, 81)),
 NerToken(word='中央健康保險署', ner='ORG', idx=(100, 107)),
 NerToken(word='今天', ner='DATE', idx=(107, 109)),
 NerToken(word='石崇良', ner='PERSON', idx=(190, 193)),
 NerToken(word='今天', ner='DATE', idx=(193, 195)),
 NerToken(word='晚間', ner='TIME', idx=(195, 197)),
 NerToken(word='5月', ner='DATE', idx=(251, 253)),
 NerToken(word='7日', ner='DATE', idx=(321, 323)),
 NerToken(word='石崇良', ner='PERSON', idx=(346, 349)),
 NerToken(word='888點', ner='CARDINAL', idx=(476, 480)),
 NerToken(word='5%', ner='PERCENT', idx=(510, 512)),
 NerToken(word='654點', ner='TIME', idx=(514, 518)),
 NerToken(word='752點', ner='TIME', idx=(520, 524)),
 NerToken(word='24.97億元', ner='MONEY', idx=(527, 534)),
 NerToken(word='石崇良', ner='PERSON', idx=(588, 591)),
 NerToken(word='10%', ner='CARDINAL', idx=(631, 634)),
 NerToken(word='30%', ner='CARDINAL', idx=(655, 658)),
 NerToken(w

In [30]:
# 過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

In [31]:
tokens_v2[0]

['健保署',
 '標準',
 '護理',
 '人員',
 '離島',
 '責任',
 '醫院',
 '醫院',
 '總額',
 '研商會',
 '新台幣',
 '衛生',
 '福利部',
 '中央',
 '保險署',
 '費用',
 '醫院',
 '總額',
 '議事會',
 '責任',
 '醫院',
 '病房',
 '離島',
 '醫院',
 '病床',
 '護理費',
 '健保署長',
 '石崇良',
 '媒體',
 '提案',
 '健保會',
 '任務',
 '指標',
 '指標',
 '醫院',
 '人員',
 '指標',
 '病房',
 '開床率',
 '健保署',
 '共識',
 '病床',
 '護理費',
 '石崇良',
 '醫學',
 '中心',
 '病床',
 '護理',
 '調幅',
 '區域',
 '醫院',
 '調幅',
 '地區',
 '醫院',
 '石崇良說',
 '醫學',
 '中心',
 '病床',
 '護理',
 '調幅',
 '區域',
 '醫院',
 '調幅',
 '地區',
 '醫院',
 '護理',
 '薪資',
 '環境',
 '護理',
 '人力',
 '患者',
 '離島',
 '醫院',
 '石崇良',
 '離島',
 '醫院',
 '標準',
 '案件',
 '定點數',
 '離島',
 '醫院',
 '案件',
 '離島',
 '醫院',
 '離島',
 '醫院',
 '能力',
 '石崇良',
 '醫師',
 '診察費',
 '護理費',
 '差異',
 '檢傷',
 '診察費',
 '護理費',
 '診察費',
 '護理費',
 '預算',
 '病房',
 '石崇良',
 '度級',
 '責任',
 '醫院',
 '病房',
 '醫學',
 '中心',
 '醫院',
 '區域',
 '醫院',
 '醫院',
 '區域',
 '醫院',
 '病人',
 '健保',
 '支出',
 '議事會',
 '台灣',
 '風險',
 '個案',
 '醫院',
 '醫院',
 '西醫基層',
 '自然',
 '手術',
 '手術',
 '石崇良',
 '醫院',
 '產科',
 '醫師',
 '點數',
 '保障',
 '微生物',
 '製劑',
 '品質',
 '計畫',
 '石崇良',
 '規定',
 '獎勵費',


In [32]:
df['tokens'] = tokens

In [33]:
df['tokens_v2'] = tokens_v2

In [34]:
df['entities'] = entity_list

In [35]:
df['token_pos'] = word_pos_pair

In [36]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[為, 解決, 急診, 壅塞, ，, 健保署, 提出, 增加, 急診, 及, 重, 症, 支...","[健保署, 標準, 護理, 人員, 離島, 責任, 醫院, 醫院, 總額, 研商會, 新台幣...","[(今天, DATE, (58, 60)), (5月, DATE, (65, 67)), (...","[(為, P), (解決, VC), (急診, VB), (壅塞, VH), (，, COM..."


In [37]:
# allowPOS過濾條件:兩個字以上與特定的詞性
# allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']
allowPOS=['Na','Nb','Nc']

from collections import Counter
def word_frequency( wp_pair ):
    filtered_words =[]
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter( filtered_words)
    return counter.most_common(20)

In [38]:
word_pos_pair[0]

[('為', 'P'),
 ('解決', 'VC'),
 ('急診', 'VB'),
 ('壅塞', 'VH'),
 ('，', 'COMMACATEGORY'),
 ('健保署', 'Nc'),
 ('提出', 'VC'),
 ('增加', 'VHC'),
 ('急診', 'VB'),
 ('及', 'Caa'),
 ('重', 'Na'),
 ('症', 'Na'),
 ('支付', 'VD'),
 ('標準', 'Na'),
 ('、', 'PAUSECATEGORY'),
 ('給', 'P'),
 ('急診', 'Nv'),
 ('護理', 'Na'),
 ('人員', 'Na'),
 ('加薪', 'VB'),
 ('、', 'PAUSECATEGORY'),
 ('強化', 'VHC'),
 ('離島', 'Nc'),
 ('急救', 'Nv'),
 ('責任', 'Na'),
 ('醫院', 'Nc'),
 ('等', 'Cab'),
 ('4', 'Neu'),
 ('招解', 'Nf'),
 ('方', 'D'),
 ('，', 'COMMACATEGORY'),
 ('醫院', 'Nc'),
 ('總額', 'Na'),
 ('研商會', 'Na'),
 ('今天', 'Nd'),
 ('通過', 'VC'),
 ('，', 'COMMACATEGORY'),
 ('最快', 'Dfa'),
 ('5月', 'Nd'),
 ('上路', 'VA'),
 ('，', 'COMMACATEGORY'),
 ('年', 'Nf'),
 ('挹注', 'VC'),
 ('逾', 'VJ'),
 ('新台幣', 'Na'),
 ('4', 'Neu'),
 ('2億', 'Neu'),
 ('元', 'Nf'),
 ('。', 'PERIODCATEGORY'),
 ('急診', 'VB'),
 ('近來', 'D'),
 ('面臨', 'VK'),
 ('前所', 'VH'),
 ('未', 'D'),
 ('有', 'V_2'),
 ('壅塞', 'VH'),
 ('，', 'COMMACATEGORY'),
 ('衛生', 'Na'),
 ('福利部', 'Na'),
 ('中央', 'Nc'),
 ('健康', 'VH'),
 ('保險署', '

In [39]:
word_frequency(word_pos_pair[0])

[('醫院', 24),
 ('離島', 7),
 ('石崇良', 7),
 ('護理', 5),
 ('護理費', 5),
 ('病房', 4),
 ('病床', 4),
 ('調幅', 4),
 ('區域', 4),
 ('人員', 3),
 ('責任', 3),
 ('指標', 3),
 ('醫學', 3),
 ('中心', 3),
 ('診察費', 3),
 ('健保署', 2),
 ('標準', 2),
 ('總額', 2),
 ('費用', 2),
 ('議事會', 2)]

In [40]:
word_frequency(word_pos_pair[1])

[('星彩', 4),
 ('號碼', 3),
 ('獎金', 3),
 ('派彩', 2),
 ('結果', 1),
 ('頭獎', 1),
 ('貳獎', 1),
 ('新台幣', 1),
 ('獎號', 1),
 ('內容', 1),
 ('台彩', 1),
 ('公司', 1)]

In [41]:
word_frequency(word_pos_pair[2])

[('星彩', 4), ('號碼', 3), ('台彩', 1), ('今彩', 1)]

In [42]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

CPU times: total: 0 ns
Wall time: 1.01 ms


In [43]:
keyfreqs[0:2]

[[('醫院', 24),
  ('離島', 7),
  ('石崇良', 7),
  ('護理', 5),
  ('護理費', 5),
  ('病房', 4),
  ('病床', 4),
  ('調幅', 4),
  ('區域', 4),
  ('人員', 3),
  ('責任', 3),
  ('指標', 3),
  ('醫學', 3),
  ('中心', 3),
  ('診察費', 3),
  ('健保署', 2),
  ('標準', 2),
  ('總額', 2),
  ('費用', 2),
  ('議事會', 2)],
 [('星彩', 4),
  ('號碼', 3),
  ('獎金', 3),
  ('派彩', 2),
  ('結果', 1),
  ('頭獎', 1),
  ('貳獎', 1),
  ('新台幣', 1),
  ('獎號', 1),
  ('內容', 1),
  ('台彩', 1),
  ('公司', 1)]]

In [44]:
df['top_key_freq'] = keyfreqs

In [45]:
df.head(2)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[為, 解決, 急診, 壅塞, ，, 健保署, 提出, 增加, 急診, 及, 重, 症, 支...","[健保署, 標準, 護理, 人員, 離島, 責任, 醫院, 醫院, 總額, 研商會, 新台幣...","[(今天, DATE, (58, 60)), (5月, DATE, (65, 67)), (...","[(為, P), (解決, VC), (急診, VB), (壅塞, VH), (，, COM...","[(醫院, 24), (離島, 7), (石崇良, 7), (護理, 5), (護理費, 5..."
1,ahel_20250312_2,2025-03-12,生活,今彩539第114064期 頭獎槓龜,今彩539第114064期開獎，中獎號碼06、14、27、30、33；派彩結果，頭獎槓龜。貳...,/news/ahel/202503120395.aspx,NaN,"[今彩, 539, 第114064, 期, 開獎, ，, 中獎, 號碼, 06, 、, 14...","[號碼, 派彩, 結果, 頭獎, 貳獎, 獎金, 新台幣, 星彩, 號碼, 獎金, 星彩, ...","[(539, CARDINAL, (2, 5)), (220注, CARDINAL, (47...","[(今彩, Nd), (539, FW), (第114064, Neu), (期, Nf),...","[(星彩, 4), (號碼, 3), (獎金, 3), (派彩, 2), (結果, 1), ..."


In [46]:
df.iloc[0].top_key_freq

[('醫院', 24),
 ('離島', 7),
 ('石崇良', 7),
 ('護理', 5),
 ('護理費', 5),
 ('病房', 4),
 ('病床', 4),
 ('調幅', 4),
 ('區域', 4),
 ('人員', 3),
 ('責任', 3),
 ('指標', 3),
 ('醫學', 3),
 ('中心', 3),
 ('診察費', 3),
 ('健保署', 2),
 ('標準', 2),
 ('總額', 2),
 ('費用', 2),
 ('議事會', 2)]

In [47]:
%%time
summary=[]
sentiment=[]
for text in df.content: # process piece by piece
    summary.append("暫無")  
    sentiment.append("暫無")

CPU times: total: 0 ns
Wall time: 0 ns


In [48]:
df['summary'] = summary

In [49]:
df['sentiment'] = sentiment

In [50]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq,summary,sentiment
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[為, 解決, 急診, 壅塞, ，, 健保署, 提出, 增加, 急診, 及, 重, 症, 支...","[健保署, 標準, 護理, 人員, 離島, 責任, 醫院, 醫院, 總額, 研商會, 新台幣...","[(今天, DATE, (58, 60)), (5月, DATE, (65, 67)), (...","[(為, P), (解決, VC), (急診, VB), (壅塞, VH), (，, COM...","[(醫院, 24), (離島, 7), (石崇良, 7), (護理, 5), (護理費, 5...",暫無,暫無


In [51]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq',
       'summary', 'sentiment'],
      dtype='object')

In [52]:
# This operation is the same as slicing
df=df[['item_id', 'date', 'category', 'title', 'content','sentiment', 'summary', 'top_key_freq', 'tokens',
       'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link'
       ]]

In [53]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,暫無,暫無,"[(醫院, 24), (離島, 7), (石崇良, 7), (護理, 5), (護理費, 5...","[為, 解決, 急診, 壅塞, ，, 健保署, 提出, 增加, 急診, 及, 重, 症, 支...","[健保署, 標準, 護理, 人員, 離島, 責任, 醫院, 醫院, 總額, 研商會, 新台幣...","[(今天, DATE, (58, 60)), (5月, DATE, (65, 67)), (...","[(為, P), (解決, VC), (急診, VB), (壅塞, VH), (，, COM...",/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...


In [54]:
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

In [55]:
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')

In [56]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,ahel_20250312_1,2025-03-12,生活,健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路,為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...,暫無,暫無,"[('醫院', 24), ('離島', 7), ('石崇良', 7), ('護理', 5),...","['為', '解決', '急診', '壅塞', '，', '健保署', '提出', '增加'...","['健保署', '標準', '護理', '人員', '離島', '責任', '醫院', '醫...","[NerToken(word='今天', ner='DATE', idx=(58, 60))...","[('為', 'P'), ('解決', 'VC'), ('急診', 'VB'), ('壅塞'...",/news/ahel/202503120382.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
